In [3]:
import numpy as np


In [4]:
def basic_sigmoid(x):
    s = 1/(1+np.exp(-x))
    return s

In [6]:
def sigmoid_derivative(x):
    # use sigmoid function derivatives
    s = basic_sigmoid(x)
    ds = s * ( 1 - s)
    return ds 
    
    

In [5]:
basic_sigmoid(np.array([1,2,3]))

array([0.73105858, 0.88079708, 0.95257413])

In [7]:
sigmoid_derivative(np.array([1,2,3]))

array([0.19661193, 0.10499359, 0.04517666])

## load in a dataset 

In [44]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 1、数据加载
iris = datasets.load_iris()

# 2、数据提取与筛选
X = iris['data']
y = iris['target']
cond = y != 2
X = X[cond]
y = y[cond]

# 3、数据拆分
X_train,X_test,y_train,y_test = train_test_split(X,y)
y_test = y_test.reshape((len(y_test),1))
y_train = y_train.reshape((len(y_train),1))

In [45]:
print("samle size of training set:  ", X_train.shape[0])
print("samle size of test set:  ", X_test.shape[0])
print("X_train shape:  ", X_train.shape)
print("y_train shape:  ", y_train.shape)
print("X_test shape:  ", X_test.shape)
print("y_test shape:  ", y_test.shape)


samle size of training set:   75
samle size of test set:   25
X_train shape:   (75, 4)
y_train shape:   (75, 1)
X_test shape:   (25, 4)
y_test shape:   (25, 1)


In [46]:
def initialize_with_zeros(shape):
    w = np.zeros([shape,1])
    b = 0
    assert(w.shape == (shape, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    return w, b

In [58]:
def propagate(w, b, X, y):
    m = X.shape[0]
    # forward propagate
    # w (n,1) X (m, n)
    A = basic_sigmoid(np.dot(X, w) + b)
    # compute loss function
    cost = -1 / m * np.sum(y * np.log(A) + (1- y) * np.log(1-A))
    # back propagate 
    # the derivative is the same form as the linear regression! 
    dz = A - y
    dw = 1 / m * np.dot(X.T, dz)
    db = 1 / m * np.sum(dz)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    grads = {"dw": dw, "db": db}
    return grads, cost 

In [59]:
def optimize(w, b, X, y, num_iterations, learning_rate):
    costs = []
    for i in range(num_iterations):
        # gradients 
        grad, cost = propagate(w, b, X, y)
        dw = grad["dw"]
        db = grad["db"]
        # update coefficients 
        w -= learning_rate * dw
        b -= learning_rate * db
        
        # record the cost change 
        if i % 100 == 0:
            costs.append(cost)
        if i % 100 == 0:
            print("results of cost %i: %f" %(i, cost))
    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    
    return params, grads, costs 

In [63]:
def predict(w, b, X):
    m = X.shape[0]
    y_predict = np.zeros((m,1))
    w = w.reshape(X.shape[1], 1)
    
    A = basic_sigmoid(np.dot(X, w) + b)
    for i in range(A.shape[0]):
        if A[i, 0] <= 0.5:
            y_predict[i, 0] = 0
        else:
            y_predict[i, 0] = 1
    assert(y_predict.shape == (m, 1))
    return y_predict 

In [64]:
def model(X_train, y_train, X_test, y_test, num_iterations= 2000, learning_rate=0.5):
    # initialize the coefficients 
    w, b = initialize_with_zeros(X_train.shape[1])
    # gradient descent 
    params, grads, costs = optimize(w, b, X_train, y_train, num_iterations, learning_rate)
    
    w = params["w"]
    b = params["b"]
    
    predict_train = predict(w, b, X_train)
    predict_test = predict(w, b, X_test)
    
    print("training set accuracy: {}".format(100-np.mean(np.abs(predict_train-y_train))*100))
    print("test set accuracy: {}".format(100-np.mean(np.abs(predict_test-y_test)*100)))
          
    
    d = {"costs": costs, 
        "y_predict_test": predict_test,
        "y_predict_train": predict_train, 
        "w": w,
        "b": b, 
        "learning_rate": learning_rate,
        "num_iterations": num_iterations}
    return d 

In [65]:
model(X_train, y_train, X_test, y_test, num_iterations= 2000, learning_rate=0.5)

results of cost 0: 0.693147
results of cost 100: 0.009987
results of cost 200: 0.006237
results of cost 300: 0.004584
results of cost 400: 0.003646
results of cost 500: 0.003037
results of cost 600: 0.002610
results of cost 700: 0.002292
results of cost 800: 0.002046
results of cost 900: 0.001850
results of cost 1000: 0.001689
results of cost 1100: 0.001556
results of cost 1200: 0.001442
results of cost 1300: 0.001345
results of cost 1400: 0.001261
results of cost 1500: 0.001187
results of cost 1600: 0.001121
results of cost 1700: 0.001063
results of cost 1800: 0.001011
results of cost 1900: 0.000963
training set accuracy: 100.0
test set accuracy: 100.0


{'costs': [0.6931471805599454,
  0.009987403151523551,
  0.006236614805497792,
  0.004584071241039865,
  0.0036456114826185163,
  0.003037460346771051,
  0.0026098570566588752,
  0.0022920154959711774,
  0.0020460315443450168,
  0.0018497340189128095,
  0.0016892687119418046,
  0.00155552213562423,
  0.0014422475425697383,
  0.001345016352869588,
  0.0012605993420181235,
  0.0011865849718416682,
  0.0011211350611686592,
  0.0010628233193762165,
  0.0010105256619204138,
  0.000963343886815173],
 'y_predict_test': array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.]]),
 'y_predict_train': array([[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
